# Text Cleaner


In [ ]:
import pandas as pd
from typing import Union
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from bs4 import BeautifulSoup
from contractions import contractions_dict


In [ ]:
def remove_repetitive_words(dataframe: pd.DataFrame, column: Union[str, int]) -> pd.DataFrame:
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_string_dtype(dataframe[column]), f"Column '{column} is not string type'"

    def remove_duplicates(text):
        if pd.isna(text):
            return text
        words = text.split()
        seen = set()
        unique_words = []
        for word in words:
            if word not in seen:
                unique_words.append(word)
                seen.add(word)
        return ' '.join(unique_words)

    df_copy = dataframe.copy()
    df_copy[column] = df_copy[column].apply(remove_duplicates)
    return df_copy


In [ ]:
def remove_stop_words(dataframe: pd.DataFrame, column: Union[str, int]):
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_string_dtype(dataframe[column]), f"Column '{column} is not string type'"
    
    df_copy = dataframe.copy()
    stop_words = set(stopwords.words('english'))
    df_copy[column] = df_copy[column].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
    return df_copy


In [ ]:
def remove_punctuation(dataframe: pd.DataFrame, column: Union[str, int]):
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_string_dtype(dataframe[column]), f"Column '{column} is not string type'"
    
    df_copy = dataframe.copy()
    df_copy[column] = df_copy[column].str.replace(r'[^\w\s]', '', regex=True)
    return df_copy


In [ ]:
def filter_words(dataframe: pd.DataFrame, column: Union[str, int], remove=['fword']):
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_string_dtype(dataframe[column]), f"Column '{column} is not string type'"
    
    df_copy = dataframe.copy()
    remove_set = set(remove)
    df_copy[column] = df_copy[column].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in remove_set]))
    return df_copy


In [ ]:
def expand_contractions(dataframe: pd.DataFrame, column: Union[str, int]):
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_string_dtype(dataframe[column]), f"Column '{column} is not string type'"
    
    contraction_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
    def expand_text(text):
        def replace(match):
            return contractions_dict[match.group(0)]
        return contraction_re.sub(replace, text)
    
    df_copy = dataframe.copy()
    df_copy[column] = df_copy[column].apply(lambda x: expand_text(x))
    return df_copy


In [ ]:
def lemmatization(dataframe: pd.DataFrame, column: Union[str, int]):
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_string_dtype(dataframe[column]), f"Column '{column} is not string type'"
    
    df_copy = dataframe.copy()
    lemmatizer = WordNetLemmatizer()
    df_copy[column] = df_copy[column].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
    return df_copy


### Test


In [ ]:
df = pd.read_csv("dataset/movies.csv")
df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')
filter_words(df, 'Title', ['gun'])
